### データフレームの準備

In [1]:
import glob
import os
import json
import pandas as pd
import shutil
import json
import numpy as np
from tqdm import tqdm

In [5]:
_df_dict = {}
_df_dict['train'] = pd.read_csv('datasets/COCOdata/train.csv')
_df_dict['test'] = pd.read_csv('datasets/COCOdata/test.csv')
_df_dict['val'] = pd.read_csv('datasets/COCOdata/val.csv')

In [6]:
_df_dict['train'][:5]

,Unnamed: 0,image_id,file_name,width,height,category_id,name,xmin,ymin,xmax,ymax
0,0,214306,000000214306.jpg,640,424,47,cup,81.60,121.90,104.57,145.07
1,1,314846,000000314846.jpg,500,375,1,person,111.62,136.10,341.33,373.73
2,2,314846,000000314846.jpg,500,375,1,person,2.20,0.00,351.13,373.90
3,3,250165,000000250165.jpg,640,426,62,chair,223.66,212.98,366.83,418.88
4,4,389624,000000389624.jpg,612,612,62,chair,78.99,321.99,135.95,415.03


### XMLファイルの作成

In [8]:
from xml.etree.ElementTree import *
import xml.etree.ElementTree as ET
import xml.dom.minidom as md
from PIL import Image

In [9]:
# 画像のサイズとチャネル数を取得
def get_image_info(path):
    img = Image.open(path)
    return img.width, img.height, img.getbands()

In [10]:
# ディレクトリとファイル名のみに分割
def file_dir_split(file_path):
    split_path = file_path.rsplit('/',1)
    dir_path = split_path[0]
    file_name = split_path[1]
    return file_name, dir_path

In [11]:
# VOCアノテーションXML ファイルを作成
def create_xml(ann_xml):
    annotation = Element('annotation')

    folder = SubElement(annotation, 'folder')
    folder.text = ann_xml['folder']

    path = SubElement(annotation, 'path')
    path.text = ann_xml['path']

    source = SubElement(annotation, 'source')
    database = SubElement(source, 'database')
    database.text = ann_xml['source']['database']

    size = SubElement(annotation, 'size')
    width = SubElement(size, 'width')
    height = SubElement(size, 'height')
    depth = SubElement(size, 'depth')
    width.text = ann_xml['size']['width']
    height.text = ann_xml['size']['height']
    depth.text = ann_xml['size']['depth']

    segmented = SubElement(annotation, 'segmented')
    segmented.text = ann_xml['segmented']

    for i, ann_object in enumerate(ann_xml['objects']):
        object = SubElement(annotation, 'object')

        name = SubElement(object, 'name')
        name.text = ann_object['name']

        pose = SubElement(object, 'pose')
        pose.text = ann_object['pose']

        truncated = SubElement(object, 'truncated')
        truncated.text = ann_object['truncated']

        difficult = SubElement(object, 'difficult')
        difficult.text = ann_object['difficult']

        bndbox = SubElement(object, 'bndbox')
        xmin = SubElement(bndbox, 'xmin')
        xmin.text = ann_object['bndbox']['xmin']
        ymin = SubElement(bndbox, 'ymin')
        ymin.text = ann_object['bndbox']['ymin']
        xmax = SubElement(bndbox, 'xmax')
        xmax.text = ann_object['bndbox']['xmax']
        ymax = SubElement(bndbox, 'ymax')
        ymax.text = ann_object['bndbox']['ymax']

    file_name, dir_path = file_dir_split(ann_xml['path'])
    file_name = file_name.split('.')[0]
    out_file_path = f'{dir_path}/{file_name}.xml'
    document = md.parseString(ET.tostring(annotation, 'utf-8'))
    with open(out_file_path, 'w') as f:
        document.writexml(f, encoding='utf-8', newl='\n', indent='', addindent='  ')

In [25]:
base_dir = 'datasets/COCOdata/COCOdata/Annotations_baobab'

In [26]:
# XMLファイルに変換
for dataset_kind, df in df_dict.items():
    file_names = df['file_name'].unique()
    i = 0
    for file_name in tqdm(file_names):
        if dataset_kind == 'train':
            if i < 56970:
                i += 1
                continue
        ann_xml = {}
        ann_xml['folder'] = dataset_kind
        ann_xml['filename'] = file_name
        ann_xml['path'] = f'{base_dir}/{ann_xml["filename"]}'
        ann_xml['source'] = {
            "database": 'Unknown'
        }
        width = df[df['file_name'] == file_name]['width'].unique().item()
        height= df[df['file_name'] == file_name]['height'].unique().item()
        depth = 3
        ann_xml['size'] = {
            "width": str(width),
            "height": str(height),
            "depth": str(depth)
        }
        ann_xml['segmented'] = '0'
        ann_xml['objects'] = []
        for index, row in df[df['file_name'] == file_name].iterrows():
            xmin = int(row['xmin'])
            ymin = int(row['ymin'])
            xmax = int(row['xmax'])
            ymax = int(row['ymax'])
            if xmin >= xmax:
                continue
                xmax += 1
                if xmax > width:
                    xmax = width
                    xmin -= 1
                print(dataset_kind)
                print(file_name)
                print(xmin, xmax)
            if ymin >= ymax:
                continue
                ymax += 1
                if ymax > height:
                    ymax = height
                    ymin -= 1
                print(dataset_kind)
                print(file_name)
                print(ymin, ymax)
            bndbox = {
                "xmin": str(xmin),
                "ymin": str(ymin),
                "xmax": str(xmax),
                "ymax": str(ymax)
            }
            object = {
                "name": row['name'],
                "pose": 'Unspecified',
                "truncated": '0',
                "difficult": '0',
                "bndbox": bndbox
            }
            ann_xml['objects'].append(object)
        create_xml(ann_xml)

100%|█████████████████████████████████████████████████████████████████████████████| 3179/3179 [00:15<00:00, 203.46it/s]


In [40]:
# データセットリストファイル train.txt, test.txt, val.txtを作成
for dataset_kind in dataset_kinds:
    xml_dir = f'{base_dir}/{dataset_kind}'
    xml_files = glob_file_names(xml_dir, 'xml')
    with open(f'{base_dir}/{dataset_kind}.txt', 'w') as f:
        for file in xml_files:
            file_name, _ = file_dir_split(file)
            file_name = file_name.split('.')[0]
            f.write(f'{file_name}\n')